In [ ]:
import pymysql
import urllib.request
import json
import time
from enum import Enum
from time import gmtime, strftime
import hashlib
import datetime as dt

from dbConfig import *

coin_name_list = ['BTC', 'ETH', 'DASH', 'LTC', 'ETC', 'XRP', 'BCH', 'XMR', 'ZEC']  # 9개 (QTUM 제외)
# coin_name_list = ['ETH', 'DASH', 'LTC', 'ETC', 'XRP', 'BCH', 'XMR', 'ZEC'] # 8개

length_process = 100

insert_trade_sql = "INSERT INTO `TRADE_{0:s}` (" + \
                   "`date`, `price`, `amount`, `total`, `type`, `exchange`, `trade_id`" + \
                   ") VALUES (%s, %s, %s, %s, %s, %s, %s);"

select_trade_sql = "SELECT count(*) FROM `TRADE_{0:s}` WHERE `trade_id`=%s"

select_in_sql='SELECT `trade_id` FROM `TRADE_{0:s}` WHERE %s'

conn = pymysql.connect(host=dbURL, port=dbPort, user=dbUser, passwd=dbPass, db=dbName, charset='utf8mb4', use_unicode=True)

cursor = conn.cursor()

class Terms_Bithumb(Enum):
    sell = "bid"
    buy = "ask"

class Terms_Poloniex(Enum):
    sell = "sell"
    buy = "buy"

def get_exist_info_for_hash_id(coin_name, api_trade_id_list):
    in_p = ' or '.join(list(map(lambda x: "trade_id='" + x + "' or trade_id='" + (x + '*') + "' or trade_id='" + (x + '**') + "'", api_trade_id_list)))
    sql = select_in_sql % in_p
    cursor.execute(sql.format(coin_name))
    conn.commit()
    return cursor.fetchall()

def get_exist_info(coin_name, api_trade_id_list):
    in_p = ' or '.join(list(map(lambda x: "trade_id='" + x + "'", api_trade_id_list)))
    sql = select_in_sql % in_p
    cursor.execute(sql.format(coin_name))
    conn.commit()
    return cursor.fetchall()

def get_trade_bithumb(coin_name):
    url_s = 'https://api.bithumb.com/public/recent_transactions/' + coin_name + '?count=' + str(length_process)
    raw_read = urllib.request.urlopen(url_s).read()
    t_bithumb = json.loads(raw_read)

    api_trade_id_list = []
    for idx in range(length_process):
        date        = t_bithumb['data'][idx]['transaction_date']
        amount      = t_bithumb['data'][idx]['units_traded']
        rate        = t_bithumb['data'][idx]['price']
        type        = Terms_Bithumb(t_bithumb['data'][idx]['type']).name
        total       = t_bithumb['data'][idx]['total']
        temp_trade_id = hashlib.sha224((date + amount + rate + type + total).encode('utf-8')).hexdigest()
        if (temp_trade_id in api_trade_id_list):
            trade_id = temp_trade_id + "*"
        elif ((temp_trade_id + "*") in api_trade_id_list):
            trade_id = temp_trade_id + "**"
        else:
            trade_id = temp_trade_id
        t_bithumb['data'][idx]['trade_id'] = trade_id
        api_trade_id_list.append(trade_id)

    exist_trade_id_list = get_exist_info_for_hash_id(coin_name, api_trade_id_list)
    exist_trade_id_list = [x[0] for x in exist_trade_id_list]
    #print('bithumb', coin_name, ', exist -', len(exist_trade_id_list))

    insert_count = 0
    for idx in range(length_process):
        date        = t_bithumb['data'][idx]['transaction_date']
        amount      = t_bithumb['data'][idx]['units_traded']
        rate        = t_bithumb['data'][idx]['price']
        type        = Terms_Bithumb(t_bithumb['data'][idx]['type']).name
        total       = t_bithumb['data'][idx]['total']
        trade_id    = t_bithumb['data'][idx]['trade_id']
        if (trade_id not in exist_trade_id_list and (trade_id is not api_trade_id_list)):
            try:
                cursor.execute(insert_trade_sql.format(coin_name), (date, rate, amount, total, type, 'bithumb', trade_id))
                insert_count += 1
            except Exception as e:
                print("Exception", e)
                pass
        conn.commit()
    return insert_count

def get_trade_poloniex(coin_name):
    url_s = 'https://poloniex.com/public?command=returnTradeHistory&currencyPair=USDT_' + coin_name + '&limit=' + str(length_process)
    raw_read = urllib.request.urlopen(url_s).read()
    t_poloniex = json.loads(raw_read)

    api_trade_id_list = []
    for idx in range(length_process):
        date        = t_poloniex[idx]['date']
        amount      = t_poloniex[idx]['amount']
        rate        = t_poloniex[idx]['rate']
        type        = Terms_Poloniex(t_poloniex[idx]['type']).name
        total       = t_poloniex[idx]['total']
        trade_id    = str(t_poloniex[idx]['globalTradeID'])
        api_trade_id_list.append(trade_id)

    exist_trade_id_list = get_exist_info(coin_name, api_trade_id_list)
    exist_trade_id_list = [x[0] for x in exist_trade_id_list]
    #print('poloniex', coin_name, ', exist -', len(exist_trade_id_list))

    insert_count = 0
    for idx in range(length_process):
        date        = t_poloniex[idx]['date']
        amount      = t_poloniex[idx]['amount']
        rate        = t_poloniex[idx]['rate']
        type        = Terms_Poloniex(t_poloniex[idx]['type']).name
        total       = t_poloniex[idx]['total']
        trade_id    = str(t_poloniex[idx]['globalTradeID'])

        if (trade_id not in exist_trade_id_list):
            try:
                cursor.execute(insert_trade_sql.format(coin_name), (date, rate, amount, total, type, 'poloniex', trade_id))
            except Exception as e:
                print("Exception", e)
                pass
            insert_count += 1
        conn.commit()

    return insert_count


if __name__ == "__main__":
    cumulative_bithumb_call_count = 0
    cumulative_poloniex_call_count = 0
    start_time = dt.datetime.now()
    while(True):
        for coin_name in coin_name_list:
            insert_count_bithumb = get_trade_bithumb(coin_name)
            insert_count_poloniex = get_trade_poloniex(coin_name)
            cumulative_bithumb_call_count += 1
            cumulative_poloniex_call_count += 1
            print("{:6s}: {:s}, New Bithumb Trade Infomation:{:3d}, New Poloniex Trade Information:{:3d}".format(
                "[" + coin_name + "]",
                strftime("%Y-%m-%d %H:%M:%S", gmtime()),
                insert_count_bithumb,
                insert_count_poloniex)
            )

        elapsed_time = (dt.datetime.now() - start_time).seconds
        print(" Bithumb API Call Rate: {:5.2f} calls/sec. (It should be less than 20 calls/sec.)".format(cumulative_bithumb_call_count / elapsed_time))
        print("Poloniex API Call Rate: {:5.2f} calls/sec. (It should be less than 6 calls/sec.).".format(cumulative_poloniex_call_count / elapsed_time))
        print()

In [1]:
import pymysql
import urllib.request
import json
import time
from enum import Enum
from time import gmtime, strftime
import hashlib
import datetime as dt

from dbConfig import *